In [1]:
# Dataset is glove 6B 50T
# udemy nlp with python

### install dependencies
only do this once

In [2]:
#! python3 -m venv venv
#! source venv/bin/activate
#! pip3 install -r requirements.txt
#! pip install ipykernel
#! python -m ipykernel install --user
#! pip freeze

### imports

In [3]:
from builtins import range
import numpy as np
import pandas as pd

from embedding_utilities import euclidean_dist, cosine_dist, load_word2vec, find_analogies, nearest_neighbors

loading word embeddings word to vec from path ../large-files/glove6B/glove.6B.50d.txt
line number 200000
soroca 0.049805 -0.92157 -0.78814 0.40001 0.73747 0.23841 1.0884 0.74811 0.35275 -0.092519 0.068657 -1.5069 0.71963 -0.62382 0.42556 -0.12594 0.12767 0.48246 0.747 0.95415 -0.61376 -0.41512 -0.04666 0.91667 -0.70671 0.71371 -0.09069 0.63914 -0.22234 -0.51031 -1.009 -0.93208 0.53891 -0.32442 0.060975 0.12915 -0.17024 0.29168 0.14596 0.18409 -0.16361 0.21103 -0.092295 -0.49671 -1.3887 0.29717 0.070581 -0.19783 -0.62638 0.25806

line number 400000
sandberger 0.072617 -0.51393 0.4728 -0.52202 -0.35534 0.34629 0.23211 0.23096 0.26694 0.41028 0.28031 0.14107 -0.30212 -0.21095 -0.10875 -0.33659 -0.46313 -0.40999 0.32764 0.47401 -0.43449 0.19959 -0.55808 -0.34077 0.078477 0.62823 0.17161 -0.34454 -0.2066 0.1323 -1.8076 -0.38851 0.37654 -0.50422 -0.012446 0.046182 0.70028 -0.010573 -0.83629 -0.24698 0.6888 -0.17986 -0.066569 -0.48044 -0.55946 -0.27594 0.056072 -0.18907 -0.59021 0.55559

tota

### the cosine distance does not change when doubeling
cosine distance only takes into account angle (the smaller the higher the similarity.)

euclidian looks at absolute length

In [4]:
a = pd.Series([1,1])
b = pd.Series([1,2])
print(f'euclidian distance {euclidean_dist(a,b)}')
print(f'cosine distance {cosine_dist(a,b)}')

euclidian distance 1.0
cosine distance 0.05131670194948623


In [5]:
a_double = a*2
b_double = b*2
print(f'euclidian distance {euclidean_dist(a_double,b_double)}')
print(f'cosine distance {cosine_dist(a_double,b_double)}')

euclidian distance 2.0
cosine distance 0.05131670194948623


In [6]:
dist, metric = cosine_dist, 'cosine'
#alternative: dist, metric = dist1, 'euclidean'

### load pretrained word vectors
source: https://www.kaggle.com/anindya2906/glove6b

In [7]:
glove_path = '../large-files/glove6B/glove.6B.50d.txt'
word2vec, embedding, index_to_word = load_word2vec(glove_path)

loading word embeddings word to vec from path ../large-files/glove6B/glove.6B.50d.txt
line number 200000
soroca 0.049805 -0.92157 -0.78814 0.40001 0.73747 0.23841 1.0884 0.74811 0.35275 -0.092519 0.068657 -1.5069 0.71963 -0.62382 0.42556 -0.12594 0.12767 0.48246 0.747 0.95415 -0.61376 -0.41512 -0.04666 0.91667 -0.70671 0.71371 -0.09069 0.63914 -0.22234 -0.51031 -1.009 -0.93208 0.53891 -0.32442 0.060975 0.12915 -0.17024 0.29168 0.14596 0.18409 -0.16361 0.21103 -0.092295 -0.49671 -1.3887 0.29717 0.070581 -0.19783 -0.62638 0.25806

line number 400000
sandberger 0.072617 -0.51393 0.4728 -0.52202 -0.35534 0.34629 0.23211 0.23096 0.26694 0.41028 0.28031 0.14107 -0.30212 -0.21095 -0.10875 -0.33659 -0.46313 -0.40999 0.32764 0.47401 -0.43449 0.19959 -0.55808 -0.34077 0.078477 0.62823 0.17161 -0.34454 -0.2066 0.1323 -1.8076 -0.38851 0.37654 -0.50422 -0.012446 0.046182 0.70028 -0.010573 -0.83629 -0.24698 0.6888 -0.17986 -0.066569 -0.48044 -0.55946 -0.27594 0.056072 -0.18907 -0.59021 0.55559

tota

In [8]:

print(embedding.shape)
embedding.head(1)

(400000, 50)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
the,0.418,0.24968,-0.41242,0.1217,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.6566,...,-0.29871,-0.15749,-0.34758,-0.045637,-0.44251,0.18785,0.002785,-0.18411,-0.11514,-0.78581


In [9]:
word2vec['tree']

array([ 0.62231 ,  1.1986  , -0.014116,  0.20125 ,  0.69419 ,  0.12068 ,
       -0.90399 , -1.4023  ,  0.43357 , -0.48537 , -0.4645  ,  0.15756 ,
        0.54261 , -0.32467 , -0.025646,  0.45742 ,  0.16561 ,  0.18819 ,
        0.062099, -0.86418 , -1.0425  , -0.81157 ,  0.3126  , -0.20279 ,
        0.55734 , -0.28634 , -0.14874 ,  1.0098  ,  0.25041 , -0.53195 ,
        2.3793  , -0.76966 , -0.63219 ,  0.3203  ,  0.15072 ,  0.23326 ,
       -0.26254 , -0.29461 ,  0.7671  , -0.11577 , -0.68129 , -0.65413 ,
       -0.58914 ,  0.24684 ,  1.5904  ,  0.33025 ,  0.41513 , -1.7468  ,
        0.82453 , -1.0886  ], dtype=float32)

In [10]:
index_to_word[123]

'country'

In [11]:
word1 = 'motivation'
word2 = 'man'
word3 = 'woman'
find_analogies(word1, word2, word3, word2vec, embedding, index_to_word, dist, metric)

motivation - man = satisfaction - woman


'satisfaction'

In [12]:
find_analogies(word1, word2, word3)#, D, index_to_word)
find_analogies('success', 'man', 'woman')#,  D, index_to_word)
find_analogies('fun', 'man', 'woman')#,  D, index_to_word)
find_analogies('love', 'woman', 'man')#,  D, index_to_word)
find_analogies('love', 'man', 'woman')#,  D, index_to_word)

find_analogies('king', 'man', 'woman')#,  D, index_to_word)
find_analogies('money', 'man', 'woman')#,  D, index_to_word)
find_analogies('sex', 'man', 'woman')#,  D, index_to_word)
find_analogies('france', 'paris', 'london')#,  D, index_to_word)
find_analogies('france', 'paris', 'rome')#,  D, index_to_word)
find_analogies('paris', 'france', 'italy')#,  D, index_to_word)
find_analogies('december', 'november', 'june')#,  D, index_to_word)
find_analogies('miami', 'florida', 'texas')#,  D, index_to_word)
find_analogies('einstein', 'scientist', 'painter')#,  D, index_to_word)
find_analogies('rice', 'china', 'germany')#,  D, index_to_word)
find_analogies('beer', 'germany', 'japan')#,  D, index_to_word)

find_analogies('tree', 'forest', 'flock')#,  D, index_to_word)

find_analogies('man', 'woman', 'she')#,  D, index_to_word)
find_analogies('man', 'woman', 'aunt')#,  D, index_to_word)
find_analogies('man', 'woman', 'sister')#,  D, index_to_word)
find_analogies('man', 'woman', 'wife')#,  D, index_to_word)
find_analogies('man', 'woman', 'friend')#,  D, index_to_word)
find_analogies('man', 'woman', 'actress')#,  D, index_to_word)
find_analogies('man', 'woman', 'mother')#,  D, index_to_word)
find_analogies('heir', 'heiress', 'princess')#,  D, index_to_word)

motivation - man = satisfaction - woman
success - man = recognition - woman
fun - man = kids - woman
love - woman = me - man
love - man = mother - woman
king - man = queen - woman
money - man = paying - woman
sex - man = sexual - woman
france - paris = britain - london
france - paris = italy - rome
paris - france = rome - italy
december - november = july - june
miami - florida = houston - texas
einstein - scientist = matisse - painter
rice - china = bacon - germany
beer - germany = drink - japan
tree - forest = devotees - flock
man - woman = he - she
man - woman = uncle - aunt
man - woman = brother - sister
man - woman = friend - wife
man - woman = brother - friend
man - woman = actor - actress
man - woman = father - mother
heir - heiress = queen - princess


'queen'

In [13]:
find_analogies('hacker', 'code', 'politician')

hacker - code = businessman - politician


'businessman'

In [14]:
nearest_neighbors('france',index_to_word, word2vec)
nearest_neighbors('japan',index_to_word, word2vec)

In [14]:
nearest_neighbors('einstein',index_to_word, word2vec)
nearest_neighbors('einstein',index_to_word, word2vec)
nearest_neighbors('woman',index_to_word, word2vec)
nearest_neighbors('man',index_to_word, word2vec)
nearest_neighbors('nephew',index_to_word, word2vec)
nearest_neighbors('february',index_to_word, word2vec)
nearest_neighbors('success',index_to_word, word2vec)
nearest_neighbors('money',index_to_word, word2vec)
nearest_neighbors('love',index_to_word, word2vec)

neighbors of: king
	prince
	queen
	ii
	emperor
	son
neighbors of: france
	french
	belgium
	paris
	spain
	netherlands
neighbors of: japan
	japanese
	china
	korea
	tokyo
	taiwan
neighbors of: einstein
	relativity
	bohr
	physics
	heisenberg
	freud
neighbors of: woman
	girl
	man
	mother
	her
	boy
neighbors of: man
	woman
	boy
	another
	old
	one
neighbors of: nephew
	cousin
	brother
	grandson
	son
	uncle
neighbors of: february
	october
	december
	january
	august
	september
neighbors of: success
	achieved
	successful
	ever
	thanks
	best
neighbors of: money
	cash
	paying
	funds
	pay
	raise
neighbors of: love
	dream
	life
	dreams
	loves
	me


array([2895,  214, 5559, 6741,  285])

In [28]:
nearest_neighbors('merkel', index_to_word, word2vec)

neighbors of: merkel
	schroeder
	sarkozy
	kohl
	steinmeier
	chirac


array([ 5886,  4767,  8168, 22720,  3924])

In [29]:
nearest_neighbors('procrastination', index_to_word, word2vec)

neighbors of: procrastination
	remedying
	rationalizing
	rationalization
	aggravation
	misjudgment


array([98759, 82245, 52093, 49897, 81264])

In [41]:
nearest_neighbors('religion', index_to_word, word2vec)

neighbors of: religion
	beliefs
	religious
	religions
	christianity
	belief


array([7107, 1213, 9847, 7282, 4440])